In [2]:
from transformers import AutoTokenizer
from transformers import TokenClassificationPipeline
from optimum.onnxruntime import ORTModelForTokenClassification
from datetime import datetime

c:\Users\zhma\Projects\.nlp\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model_id = './checkpoint-9360'
tokenizer = AutoTokenizer.from_pretrained(model_id)
model_onnx = ORTModelForTokenClassification.from_pretrained(model_id, export=True)

Framework not specified. Using pt to export the model.


Using the export variant default. Available variants are:
    - default: The default ONNX variant.

***** Exporting submodel 1/1: DistilBertForTokenClassification *****
Using framework PyTorch: 2.4.0+cu118
c:\Users\zhma\Projects\.nlp\Lib\site-packages\transformers\models\distilbert\modeling_distilbert.py:215: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  mask, torch.tensor(torch.finfo(scores.dtype).min)


In [4]:
onnx_path = 'wfu_model_distill_onnx'
model_onnx.save_pretrained(onnx_path)


In [5]:
model_onnx.device

device(type='cpu')

In [6]:
from optimum.onnxruntime import ORTOptimizer
from optimum.onnxruntime.configuration import OptimizationConfig

In [7]:
optimizer = ORTOptimizer.from_pretrained(onnx_path)
optimization_config = OptimizationConfig(optimization_level=99,
                                         optimize_for_gpu=True,
                                         fp16=True
                                         )

c:\Users\zhma\Projects\.nlp\Lib\site-packages\optimum\onnxruntime\configuration.py:779: FutureWarning: disable_embed_layer_norm will be deprecated soon, use disable_embed_layer_norm_fusion instead, disable_embed_layer_norm_fusion is set to True.
  warnings.warn(


In [8]:
optimizer.optimize(optimization_config, save_dir='wfu_model_onnx_optimized')

Optimizing model...
Failed to remove node input: "/distilbert/transformer/layer.0/attention/Transpose_output_0"
input: "/distilbert/transformer/layer.0/attention/Constant_11_output_0"
output: "/distilbert/transformer/layer.0/attention/Div_output_0"
name: "/distilbert/transformer/layer.0/attention/Div"
op_type: "Div"

Failed to remove node input: "/distilbert/transformer/layer.1/attention/Transpose_output_0"
input: "/distilbert/transformer/layer.1/attention/Constant_11_output_0"
output: "/distilbert/transformer/layer.1/attention/Div_output_0"
name: "/distilbert/transformer/layer.1/attention/Div"
op_type: "Div"

Failed to remove node input: "/distilbert/transformer/layer.2/attention/Transpose_output_0"
input: "/distilbert/transformer/layer.2/attention/Constant_11_output_0"
output: "/distilbert/transformer/layer.2/attention/Div_output_0"
name: "/distilbert/transformer/layer.2/attention/Div"
op_type: "Div"

Failed to remove node input: "/distilbert/transformer/layer.3/attention/Transpose_o

WindowsPath('wfu_model_onnx_optimized')

In [9]:
TokenClassificationPipeline?

Init signature:
TokenClassificationPipeline(
    args_parser=<transformers.pipelines.token_classification.TokenClassificationArgumentHandler object at 0x000002642AAB4090>,
    *args,
    **kwargs,
)
Docstring:     
Named Entity Recognition pipeline using any `ModelForTokenClassification`. See the [named entity recognition
examples](../task_summary#named-entity-recognition) for more information.

Example:

```python
>>> from transformers import pipeline

>>> token_classifier = pipeline(model="Jean-Baptiste/camembert-ner", aggregation_strategy="simple")
>>> sentence = "Je m'appelle jean-baptiste et je vis à montréal"
>>> tokens = token_classifier(sentence)
>>> tokens
[{'entity_group': 'PER', 'score': 0.9931, 'word': 'jean-baptiste', 'start': 12, 'end': 26}, {'entity_group': 'LOC', 'score': 0.998, 'word': 'montréal', 'start': 38, 'end': 47}]

>>> token = tokens[0]
>>> # Start and end provide an easy way to highlight words in the original text.
>>> sentence[token["start"] : token["end"]]
'

In [17]:
model_onnx_optimized = ORTModelForTokenClassification.from_pretrained('wfu_model_onnx_optimized', 
                                                                      export=False,
                                                                      provider='CUDAExecutionProvider')

In [1]:
clf = TokenClassificationPipeline(model=model_onnx_optimized, tokenizer=tokenizer, device=0)

NameError: name 'TokenClassificationPipeline' is not defined

In [19]:
with open('actual_note_test.txt', 'r') as fid:
    line = fid.read()
all_texts = line.split('---')

In [21]:
print(datetime.now())
results = clf(all_texts, stride=16, ignore_labels=['NORMAL'], aggregation_strategy='max', batch_size=16)
print(datetime.now())

2024-08-21 10:28:15.893396
2024-08-21 10:35:11.993453


In [22]:
results[:-5]

[[],
 [{'entity_group': 'NAME',
   'score': 0.9997521,
   'word': 'aguilar',
   'start': 68,
   'end': 75}],
 [{'entity_group': 'NAME',
   'score': 0.9998142,
   'word': 'richard j preske',
   'start': 138,
   'end': 154}],
 [{'entity_group': 'HOSPITAL',
   'score': 0.99969894,
   'word': 'wake forest baptist health brenner children & apos ; s hospital pediatric otolaryngology clinic',
   'start': 0,
   'end': 91},
  {'entity_group': 'DATE',
   'score': 0.9998263,
   'word': '3 / 2 / 2015',
   'start': 108,
   'end': 116},
  {'entity_group': 'NAME',
   'score': 0.9998072,
   'word': 'anthony felicianojuarez',
   'start': 118,
   'end': 141},
  {'entity_group': 'IDNUM',
   'score': 0.9997845,
   'word': '2012018',
   'start': 151,
   'end': 158},
  {'entity_group': 'DATE',
   'score': 0.9998059,
   'word': '12 / 18 / 2007',
   'start': 165,
   'end': 175},
  {'entity_group': 'NAME',
   'score': 0.99980766,
   'word': 'anthony',
   'start': 178,
   'end': 185},
  {'entity_group': 'AGE',


In [23]:
model_onnx_optimized = ORTModelForTokenClassification.from_pretrained('wfu_model_onnx_optimized', 
                                                                      export=False,
                                                                      provider='CUDAExecutionProvider',
                                                                      use_io_binding=True)

In [24]:
clf = TokenClassificationPipeline(model=model_onnx_optimized, tokenizer=tokenizer, device=0)

In [25]:
print(datetime.now())
results = clf(all_texts, stride=16, ignore_labels=['NORMAL'], aggregation_strategy='max', batch_size=16)
print(datetime.now())

2024-08-21 10:37:52.594883
2024-08-21 10:44:17.228796
